# Imports

In [16]:
import duckdb
import sqlite3
import pandas as pd
import re
import requests
import matplotlib.pyplot as plt
import sys
import re

import numpy as np
import pandas as pd

import plotly.express as px
import cufflinks as cf
cf.go_offline()

import statistics

import seaborn as sns

# Read diamonds.db

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',None)

In [18]:
#Import with sqlite3 the diamonds_train tables
conn = sqlite3.connect("../data/diamonds_train.db")
diamonds_city = pd.read_sql_query("SELECT * FROM diamonds_city", conn)
diamonds_clarity = pd.read_sql_query("SELECT * FROM diamonds_clarity", conn)
diamonds_color = pd.read_sql_query("SELECT * FROM diamonds_color", conn)
diamonds_cut = pd.read_sql_query("SELECT * FROM diamonds_cut", conn)
diamonds_dimensions = pd.read_sql_query("SELECT * FROM diamonds_dimensions", conn)
diamonds_properties = pd.read_sql_query("SELECT * FROM diamonds_properties", conn)
diamonds_transactional = pd.read_sql_query("SELECT * FROM diamonds_transactional", conn)

# Merge diamonds.db

In [19]:
#Merge the dataframes by the different id's
df1 = pd.merge(pd.merge(diamonds_dimensions, diamonds_properties, on='index_id'), diamonds_transactional, on='index_id')
df2 = pd.merge(df1, diamonds_cut, on = 'cut_id')
df3 = pd.merge(df2, diamonds_color, on = 'color_id')
df4 = pd.merge(df3, diamonds_clarity, on = 'clarity_id')
df5 = pd.merge(df4, diamonds_city, on = 'city_id')

In [20]:
#Keep only the columns that I need
df6 = df5[["depth", "table", "x", "y", "z", "price", "carat", "cut", "color", "clarity"]]
diamonds_train = df6[["price", "carat", "cut", "color", "clarity", "depth", "table", "x", "y", "z"]]
diamonds_train_sorted = diamonds_train.sort_values(by='price').reset_index(drop=True)
diamonds_train_sorted.head()

,price,carat,cut,color,clarity,depth,table,x,y,z
0,326,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
1,326,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
2,327,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,334,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,335,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


In [21]:
"""
Explanation of the diamonds Features:
price: price in USD
carat: weight of the diamond
cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
color: diamond colour, from J (worst) to D (best)
clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
x: length in mm
y: width in mm
z: depth in mm
depth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
table: width of top of diamond relative to widest point (43--95)
city: city where the diamonds is reported to be sold.
"""

'\nExplanation of the diamonds Features:\nprice: price in USD\ncarat: weight of the diamond\ncut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)\ncolor: diamond colour, from J (worst) to D (best)\nclarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))\nx: length in mm\ny: width in mm\nz: depth in mm\ndepth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)\ntable: width of top of diamond relative to widest point (43--95)\ncity: city where the diamonds is reported to be sold.\n'

# Label encoding diamonds.db

In [22]:
#Label Encoding
cut_encoding = {'Premium': 4, 'Very Good': 3, 'Fair': 2, 'Good': 1, 'Ideal': 0}
def cut_label_encoding(x):
    for key in cut_encoding:
        if x == key:
            return cut_encoding[key]

In [23]:
diamonds_train_sorted['cut'] = diamonds_train_sorted['cut'].apply(cut_label_encoding)
diamonds_train_sorted

,price,carat,cut,color,clarity,depth,table,x,y,z
0,326,0.21,4,E,SI1,59.8,61.0,3.89,3.84,2.31
1,326,0.23,0,E,SI2,61.5,55.0,3.95,3.98,2.43
2,327,0.23,1,E,VS1,56.9,65.0,4.05,4.07,2.31
3,334,0.29,4,I,VS2,62.4,58.0,4.20,4.23,2.63
4,335,0.31,1,J,SI2,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
40450,18795,2.04,4,H,SI1,58.1,60.0,8.37,8.28,4.84
40451,18797,2.29,4,I,SI1,61.8,59.0,8.52,8.45,5.24
40452,18806,1.51,0,G,IF,61.7,55.0,7.37,7.41,4.56
40453,18818,2.00,3,G,SI1,63.5,56.0,7.90,7.97,5.04


In [24]:
color_encoding = {'J': 6, 'I': 5, 'H': 4, 'G': 3, 'F': 2, 'E': 1, 'D': 0}
def color_label_encoding(x):
    for key in color_encoding:
        if x == key:
            return color_encoding[key]

In [25]:
diamonds_train_sorted['color'] = diamonds_train_sorted['color'].apply(color_label_encoding)
diamonds_train_sorted

,price,carat,cut,color,clarity,depth,table,x,y,z
0,326,0.21,4,1,SI1,59.8,61.0,3.89,3.84,2.31
1,326,0.23,0,1,SI2,61.5,55.0,3.95,3.98,2.43
2,327,0.23,1,1,VS1,56.9,65.0,4.05,4.07,2.31
3,334,0.29,4,5,VS2,62.4,58.0,4.20,4.23,2.63
4,335,0.31,1,6,SI2,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
40450,18795,2.04,4,4,SI1,58.1,60.0,8.37,8.28,4.84
40451,18797,2.29,4,5,SI1,61.8,59.0,8.52,8.45,5.24
40452,18806,1.51,0,3,IF,61.7,55.0,7.37,7.41,4.56
40453,18818,2.00,3,3,SI1,63.5,56.0,7.90,7.97,5.04


In [26]:
clarity_encoding = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
def clarity_label_encoding(x):
    for key in clarity_encoding:
        if x == key:
            return clarity_encoding[key]

In [27]:
diamonds_train_sorted['clarity'] = diamonds_train_sorted['clarity'].apply(clarity_label_encoding)
diamonds_train_sorted

,price,carat,cut,color,clarity,depth,table,x,y,z
0,326,0.21,4,1,2,59.8,61.0,3.89,3.84,2.31
1,326,0.23,0,1,1,61.5,55.0,3.95,3.98,2.43
2,327,0.23,1,1,4,56.9,65.0,4.05,4.07,2.31
3,334,0.29,4,5,3,62.4,58.0,4.20,4.23,2.63
4,335,0.31,1,6,1,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
40450,18795,2.04,4,4,2,58.1,60.0,8.37,8.28,4.84
40451,18797,2.29,4,5,2,61.8,59.0,8.52,8.45,5.24
40452,18806,1.51,0,3,7,61.7,55.0,7.37,7.41,4.56
40453,18818,2.00,3,3,2,63.5,56.0,7.90,7.97,5.04


# Dropping dimentionless diamonds

In [28]:

diamonds_train_sorted = diamonds_train_sorted.drop(diamonds_train_sorted[diamonds_train_sorted["x"] == 0].index)
diamonds_train_sorted = diamonds_train_sorted.drop(diamonds_train_sorted[diamonds_train_sorted["y"] == 0].index)
diamonds_train_sorted = diamonds_train_sorted.drop(diamonds_train_sorted[diamonds_train_sorted["z"] == 0].index)


# Dropping the outliers

In [29]:

diamonds_train_sorted = diamonds_train_sorted[(diamonds_train_sorted["depth"]<75)&(diamonds_train_sorted["depth"]>45)]
diamonds_train_sorted = diamonds_train_sorted[(diamonds_train_sorted["table"]<80)&(diamonds_train_sorted["table"]>40)]
diamonds_train_sorted = diamonds_train_sorted[(diamonds_train_sorted["x"]<30)]
diamonds_train_sorted = diamonds_train_sorted[(diamonds_train_sorted["y"]<30)]
diamonds_train_sorted = diamonds_train_sorted[(diamonds_train_sorted["z"]<30)&(diamonds_train_sorted["z"]>2)]


In [40]:
diamonds_train_sorted.to_csv('../data/diamonds_train_sorted.csv', index=False)

# Read diamonds_test

In [30]:
diamonds_test = pd.read_csv('../data/diamonds_test.csv')
diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


# Label encoding diamonds_test

In [31]:
#Label Encoding
cut_encoding = {'Premium': 4, 'Very Good': 3, 'Fair': 2, 'Good': 1, 'Ideal': 0}
def cut_label_encoding(x):
    for key in cut_encoding:
        if x == key:
            return cut_encoding[key]

In [32]:
diamonds_test['cut'] = diamonds_test['cut'].apply(cut_label_encoding)

In [33]:
color_encoding = {'J': 6, 'I': 5, 'H': 4, 'G': 3, 'F': 2, 'E': 1, 'D': 0}
def color_label_encoding(x):
    for key in color_encoding:
        if x == key:
            return color_encoding[key]

In [34]:
diamonds_test['color'] = diamonds_test['color'].apply(color_label_encoding)

In [35]:
clarity_encoding = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
def clarity_label_encoding(x):
    for key in clarity_encoding:
        if x == key:
            return clarity_encoding[key]

In [36]:
diamonds_test['clarity'] = diamonds_test['clarity'].apply(clarity_label_encoding)

In [37]:
diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,3,2,2,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,0,6,4,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,4,4,2,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,3,2,2,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,3,2,4,62.9,58.0,5.05,5.09,3.19,Amsterdam


In [38]:
diamonds_test_processed = diamonds_test[["carat", "cut", "color", "clarity", "depth", "table", "x", "y", "z"]]
diamonds_test_processed.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.79,3,2,2,62.7,60.0,5.82,5.89,3.67
1,1.20,0,6,4,61.0,57.0,6.81,6.89,4.18
2,1.57,4,4,2,62.2,61.0,7.38,7.32,4.57
3,0.90,3,2,2,63.8,54.0,6.09,6.13,3.90
4,0.50,3,2,4,62.9,58.0,5.05,5.09,3.19


# Turn the zero values

In [39]:
def remove_column_zero(col):
    for i in diamonds_test_processed[col].index:
        if diamonds_test_processed[col][i]==0:
            diamonds_test_processed[col][i] = diamonds_test_processed[col].median()
remove_column_zero('x')
remove_column_zero('y')
remove_column_zero('z')
diamonds_test_processed.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.79,3,2,2,62.7,60.0,5.82,5.89,3.67
1,1.20,0,6,4,61.0,57.0,6.81,6.89,4.18
2,1.57,4,4,2,62.2,61.0,7.38,7.32,4.57
3,0.90,3,2,2,63.8,54.0,6.09,6.13,3.90
4,0.50,3,2,4,62.9,58.0,5.05,5.09,3.19


# Dropping dimentionless diamonds

In [25]:
"""
diamonds_test_processed = diamonds_test_processed.drop(diamonds_test_processed[diamonds_test_processed["x"] == 0].index)
diamonds_test_processed = diamonds_test_processed.drop(diamonds_test_processed[diamonds_test_processed["y"] == 0].index)
diamonds_test_processed = diamonds_test_processed.drop(diamonds_test_processed[diamonds_test_processed["z"] == 0].index)
"""

'\ndiamonds_test_processed = diamonds_test_processed.drop(diamonds_test_processed[diamonds_test_processed["x"] == 0].index)\ndiamonds_test_processed = diamonds_test_processed.drop(diamonds_test_processed[diamonds_test_processed["y"] == 0].index)\ndiamonds_test_processed = diamonds_test_processed.drop(diamonds_test_processed[diamonds_test_processed["z"] == 0].index)\n'

# Dropping the outliers

In [26]:
"""
diamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["depth"]<75)&(diamonds_test_processed["depth"]>45)]
diamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["table"]<80)&(diamonds_test_processed["table"]>40)]
diamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["x"]<30)]
diamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["y"]<30)]
diamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["z"]<30)&(diamonds_test_processed["z"]>2)]
"""

'\ndiamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["depth"]<75)&(diamonds_test_processed["depth"]>45)]\ndiamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["table"]<80)&(diamonds_test_processed["table"]>40)]\ndiamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["x"]<30)]\ndiamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["y"]<30)]\ndiamonds_test_processed = diamonds_test_processed[(diamonds_test_processed["z"]<30)&(diamonds_test_processed["z"]>2)]\n'

In [41]:
diamonds_test_processed.to_csv('../data/diamonds_test_processed.csv', index=False)